In [6]:
import numpy as np
a=np.sum([1,2,3])
type(float(a/a))

float

In [ ]:
from typing import*
def f(n: int) -> bool:
    """Determine if a given integer 'n' is a power of two."""
    return n > 0 and (n & (n - 1)) == 0
def g():
    return 16
assert f(g())

In [ ]:
n=16
n & (n - 1)

In [ ]:
['elm', 'elm_NLP', 'imgep_random', 'imgep_smart', 'rd_gen']
path="/home/flowers/project/evaluate_model/save_results/elmopen_llama_3b_v2_e2_seed_21.json"
pass_k=[
    {
        "pass_1": 159.2,
        "pass_2": 235.97777777777776,
        "pass_3": 283.975,
        "pass_4": 317.9666666666667,
        "pass_5": 343.82142857142856,
        "pass_6": 364.4238095238095,
        "pass_7": 381.4166666666667,
        "pass_8": 395.79999999999995,
        "pass_9": 408.20000000000005,
        "pass_10": 419.0
    }
]

In [ ]:
import pickle
path="/home/flowers/project/evaluate_model/save_feat/.WizardCoder-1B-V1.0_feat.pkl.Dz0o9L"
with open(path, 'rb') as f:
    feat_dict = pickle.load(f)

rd_gen
3
elm
3
elm_NLP
1 3
imgep_random
1 3
imgep_smart
3


In [ ]:
e

In [ ]:
import ast
import re
import os
def just_remove_example_in_docstring(source_code: str) -> str:
    puzzle_formated= source_code

    # Parse the source code into an AST
    tree = ast.parse(source_code)

    # Extract the docstring from function f and remove it
    f_docstring = None
    for item in tree.body:
        if isinstance(item, ast.FunctionDef) and item.name == 'f':
            if ast.get_docstring(item):
                f_docstring = ast.get_docstring(item)
                if (f_docstring != None):
                    delimiters ="example","Example","For example","Example:"
                    regex_pattern = '|'.join(map(re.escape, delimiters))
                    f_docstring_split = re.split(regex_pattern, f_docstring)[0]
                    item.body[0].value.s = f_docstring_split
    if (f_docstring != None):
        # Convert the modified AST back to source code
        puzzle_formated=ast.unparse(tree)
    puzzle_formated=puzzle_formated.replace('""""""',"")
    puzzle_formated = os.linesep.join([s for s in puzzle_formated.splitlines() if s.strip()]) # remove empty line

    return puzzle_formated

In [ ]:
import json
path="/home/flowers/project/evaluate_model/save_feat/P3_trainWizardCoder-1B-V1.0.json"
with open(path, 'r') as f:
    data = json.load(f)
len(data),len(data[0]["emb_features"])


In [ ]:
import json
from datasets import load_dataset,concatenate_datasets
from utils_test import remove_example_line,prompt_solve_puzzle

path= "/home/flowers/project/evaluate_model/run_saved/maps_2_imgep_smart.json"
print(path)
with open(path, 'r') as f:
    data = json.load(f)
list_prg=[puz["program_str"] for puz in data]
dataset_r = load_dataset("json", data_files=path, split="train")
output_texts=[]
for i in range(len(dataset_r['program_str'])):
    try:
        puzzle= just_remove_example_in_docstring(dataset_r["program_str"][i])
        prompt_f=puzzle.split("def g(")[0]
        prompt_g= "def g(" + puzzle.split("def g(")[1]
        full_prompt = prompt_solve_puzzle.format(pb=prompt_f,g_firstline=prompt_g)
        output_texts.append(full_prompt)
    except:
        print(i)


In [ ]:
dataset_r["program_str"][i]

In [ ]:
print(dataset_r["program_str"][i])
print("===============")
remove_example_line(dataset_r["program_str"][i])

In [ ]:
import json
from datasets import load_dataset,concatenate_datasets
from utils_test import remove_example_line

path= "run_saved/maps_1_imgep_random.json"
print(path)
with open(path, 'r') as f:
    data = json.load(f)
list_prg=[puz["program_str"] for puz in data]
dataset_r = load_dataset("json", data_files=path, split="train")
list_aa=[]
for i in range(len(dataset_r['program_str'])):
    
    puzzle= remove_example_line(dataset_r["program_str"][i])
    list_aa.append(puzzle)

In [ ]:
puz="from typing import*\nfrom typing import List\n\ndef f(nums: List[int]) -> int:\n    \"\"\"Find the sum of the squares of the three largest numbers in the given list\"\"\"\n    sorted_nums = sorted(nums, reverse=True)\n    return sum([num**2 for num in sorted_nums[:3]])\n\ndef g():\n    return [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]\n\nassert f(g()) == 245\n"
print(puz)

In [ ]:
from utils_test import pass_at_k,prompt_solve_puzzle
model_id = "openlm-research/open_llama_3b_v2"
tokenizer = LlamaTokenizer.from_pretrained(model_id,local_files_only=True)

def formatting_prompts_func(example,prompt_solve_puzzle=prompt_solve_puzzle):
    output_texts = []
    # print(len(example['program_str']))
    for i in range(len(example['program_str'])):
        if True:
            puzzle= remove_example_line(example['program_str'][i])
        else:
            puzzle= example['program_str'][i]

        prompt_f=puzzle.split("def g(")[0]
        prompt_g= "def g(" + puzzle.split("def g(")[1]
        full_prompt = prompt_solve_puzzle.format(pb=prompt_f,g_firstline=prompt_g)
        output_texts.append(full_prompt)
    return output_texts

def _prepare_non_packed_dataloader(
    tokenizer, dataset, dataset_text_field, max_seq_len, formatting_func=None
):
    use_formatting_func = formatting_func is not None and dataset_text_field is None

    # Inspired from: https://huggingface.co/learn/nlp-course/chapter7/6?fw=pt
    def tokenize(element):
        outputs = tokenizer(
            formatting_func(element),
            truncation=True,
            padding=False,
            max_length=max_seq_len,
            return_overflowing_tokens=False,
            return_length=False,
        )

        if use_formatting_func:
            if not isinstance(formatting_func(element), list):
                raise ValueError(
                    "The `formatting_func` should return a list of processed strings since it can lead to silent bugs."
                )
        return {"input_ids": outputs["input_ids"], "attention_mask": outputs["attention_mask"]}

    tokenized_dataset = dataset.map(
        tokenize,
        batched=True,
        remove_columns=dataset.column_names,
        num_proc=2,
        batch_size=8,
    )

    return tokenized_dataset

In [ ]:
_prepare_non_packed_dataloader(

# sample puzzles 2 label

In [ ]:
import numpy as np
import json
import matplotlib.pyplot as plt
path = "/home/flowers/project/evaluate_model/run_saved/maps_1_imgep_smart.json"
with open(path, 'r') as f:
    data = json.load(f)
list_prg=[puz["program_str"] for puz in data]
for puzz in data:
    
    emb=np.array(puzz["emb"],dtype=int).tolist()
    puzz["emb"]=emb

In [ ]:
import numpy as np
import json
import copy
import matplotlib.pyplot as plt
rng = np.random.default_rng(seed=42)
list_puzzle_2_label=[]
list_label=np.array([0. for _ in range((10))])

path = "/home/flowers/project/evaluate_model/run_saved/maps_1_imgep_smart.json"
with open(path, 'r') as f:
    data = json.load(f)
list_prg=[puz["program_str"] for puz in data]
list_emb=[puz["emb"] for puz in data]
idx_uniform=rng.choice(len(data), 30, replace=False)
print(len(data))
for idx_unif in idx_uniform:
    list_puzzle_2_label.append(copy.deepcopy(data[idx_unif]))
    list_label-=np.array(data[idx_unif]["emb"])
    del data[idx_unif]
print(len(data))

for idx_del,puzz in enumerate(data):
    
    emb=np.array(puzz["emb"],dtype=int).tolist()
    puzz["emb"]=emb
    if np.sum(emb)==10:
        del data[idx_del]
print(len(data))

idx_shuffle=np.array([i for i in range(len(data))])
rng.shuffle(idx_shuffle)
while len(list_puzzle_2_label)<60:
    for choosen_idx in idx_shuffle:
        if len(list_puzzle_2_label)>=60:
            break
        idx2aim = np.argmax(list_label)
        puzzle=data[choosen_idx]
        emb=puzzle["emb"]
        if emb[idx2aim]==1:
            list_puzzle_2_label.append(puzzle)
            list_label-=np.array(emb)
list_label


In [ ]:
plt.bar([i for i in range(10)],-list_label)
plt.xlabel("skill idx")
plt.ylabel("number of puzzle")

In [ ]:
emb_2=[ puzz["emb"] for puzz in list_puzzle_2_label]

In [ ]:

plt.hist((np.sum(emb_2,axis=1)),bins=100)

In [ ]:
path_puzzles="/home/flowers/project/evaluate_model/subset2label.json"
list_results=[]

    
with open(path_puzzles) as f:
        archive=json.load(f)
list_emb =[item["emb"] for item in archive]
count=0
print((np.sum(list_emb,axis=1)==10).sum())

In [ ]:
plt.hist(np.sum(list_emb,axis=1),bins=13)

In [ ]:
list_puzzle_2_label
path = "/home/flowers/project/evaluate_model/subset2label.json"
with open(path, 'w') as f:
    json.dump(list_puzzle_2_label,f,indent=4)



# label puzzles

In [ ]:
list_puzzle_2_label
path = "/home/flowers/project/evaluate_model/subset2label.json"
with open(path, 'r') as f:
    data2label = json.load(f)

In [ ]:
data2label[0]

In [ ]:
1155    1472   
1406    1268  
1117

In [ ]:
for idx in range(len(data2label)):
    if not ("GT_emb" in data2label[idx].keys()):
        print(data2label[idx]["program_str"])
        
        data2label[idx]["GT_emb"]

    data2label["program_str"]

In [ ]:
import numpy as np

truth_labels = [
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 1]
]

detected_labels = [
 [0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 1, 1, 0],
 [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 1, 0, 0, 1, 0]
]



# number of labels
n_labels = len(truth_labels[0])

# initialize matrix
matrix = np.zeros((n_labels, n_labels))

# count for each ground truth label
label_counts = np.zeros(n_labels)



for truth, detected in zip(truth_labels, detected_labels):
    for i in range(n_labels):
        if truth[i] == 1:
            label_counts[i] += 1
            for j in range(n_labels):
                if detected[j] == 1:
                    matrix[i][j] += 1

# Normalize
for i in range(n_labels):
    if label_counts[i] != 0:
        matrix[i] = matrix[i] / label_counts[i]

print(matrix)


In [ ]:
path_rd="/home/flowers/project/evaluate_model/run_saved/maps_1_rd_gen.json"
path_elm="/home/flowers/project/evaluate_model/run_saved/maps_1_elm.json"
path_elm_nl="/home/flowers/project/evaluate_model/run_saved/maps_1_elm_NLP.json"
path_imrd = "/home/flowers/project/evaluate_model/run_saved/maps_1_imgep_random.json"
path_imgep = "/home/flowers/project/evaluate_model/run_saved/maps_1_imgep_smart.json"

list_path = [path_rd,path_elm,path_elm_nl,path_imrd,path_imgep]
name=["rd","elm","elm_nl","imrd","imgep"]
list_res=[]
for i,path in enumerate(list_path):
    with open(path, 'r') as f:
        data = json.load(f)
    list_prg=[puz["program_str"] for puz in data]


In [ ]:
import itertools
def generate_vectors(n):
    return [list(p) for p in itertools.product([0, 1], repeat=n)]

vectors = generate_vectors(10)
for vector in vectors:
    print(vector)
dic={str(vector): [] for vector in vectors}